# Sentiment Analysis
The sentiment analysis of the tweets. <br />
Training done on dataset on twitter at https://www.kaggle.com/c/tweet-sentiment-extraction

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('data/tweet-sentiment-extraction/train.csv')
df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [3]:
train = df.loc[:, ['text', 'sentiment']]
train.head()

,text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD I will miss you here in San Diego!!!,negative
2,my boss is bullying me...,negative
3,what interview! leave me alone,negative
4,"Sons of ****, why couldn`t they put them on t...",negative


# Using the VADER SentimentIntensityAnalyzer
Let's see the performance of Vader on scraped tweets.

In [4]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [5]:
sentiment_analyser = SentimentIntensityAnalyzer()

In [6]:
actual = []
pred = []
for row in train.iterrows():
    text = row[1]['text']
    
    sentiment = sentiment_analyser.polarity_scores(str(text))
    sentiment['compound'] = sentiment['compound']
    
    predicted = ''
    if sentiment['compound'] >= 0.05:
        predicted = 'positive'
    elif sentiment['compound'] <= -0.05:
        predicted = 'negative'
    else:
        predicted = 'neutral'
        
        
    actual.append(row[1]['sentiment'])
    pred.append(predicted)

In [7]:
actual = np.array(actual)
pred = np.array(pred)

In [8]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [9]:
cm = confusion_matrix(actual, pred)
print(cm)

[[4638 1426 1717]
 [1664 5245 4209]
 [ 340  779 7463]]


In [10]:
acc = accuracy_score(actual, pred)
print(acc)

0.6311997380008005


In [11]:
report = classification_report(actual, pred)
print(report)

              precision    recall  f1-score   support

    negative       0.70      0.60      0.64      7781
     neutral       0.70      0.47      0.56     11118
    positive       0.56      0.87      0.68      8582

    accuracy                           0.63     27481
   macro avg       0.65      0.65      0.63     27481
weighted avg       0.66      0.63      0.62     27481



# Using Deep Learning
Let's now train a deep learning model and see it's performance on scraped tweets.